In [15]:
import pickle
import os
import numpy as np
import pandas as pd
import tensorflow as tf

from keras.layers import Dense, Dropout, Embedding
from keras.models import Input, Model
from keras.preprocessing import sequence
from tensorflow.python.framework import ops
from tcn import TCN
from sklearn.model_selection import train_test_split
from keras.models import load_model



ops.reset_default_graph()

In [2]:
data = pickle.load( open( "dfsm.pkl", "rb" ) )
cols=list(data[1].iloc[:,:34].columns)+list(data[1].iloc[:,-6:-1])
x=list()
y=list()
for i in range(len(data)):
    x.append(np.asarray(data[i][cols[:35]]))
    y.append(np.asarray(data[i].iloc[:,-1:]))
        
x_norm=list()
for i in range(len(x)):
    x_norm.append((x[i]+0.000000000000000000001)/(x[i].max(axis=0)+0.000000000000000000001))

In [3]:
x_train, x_test, y_train, y_test = train_test_split(x_norm, y, test_size=0.3,random_state=30)

In [4]:
def get_x_y_shift(x_train,y_train):
    while True:
        for k in range(len(x_train)):
            x_t = x_train[k]
            y_t = y_train[k]
            sample_weights=5*(y_t)+1
            yield (np.expand_dims(np.array(x_t),axis=0), np.expand_dims(np.array(y_t),axis=0),sample_weights.T)

In [5]:
gen = get_x_y_shift(x_train=x_train,y_train=y_train)

In [6]:
def compute_prediction_utility(labels, predictions, dt_early=-12, dt_optimal=-6, dt_late=3.0, max_u_tp=1, min_u_fn=-2, u_fp=-0.05, u_tn=0, check_errors=True):
    # Check inputs for errors.
    if check_errors:
        if len(predictions) != len(labels):
            raise Exception('Numbers of predictions and labels must be the same.')

        for label in labels:
            if not label in (0, 1):
                raise Exception('Labels must satisfy label == 0 or label == 1.')

        for prediction in predictions:
            if not prediction in (0, 1):
                raise Exception('Predictions must satisfy prediction == 0 or prediction == 1.')

        if dt_early >= dt_optimal:
            raise Exception('The earliest beneficial time for predictions must be before the optimal time.')

        if dt_optimal >= dt_late:
            raise Exception('The optimal time for predictions must be before the latest beneficial time.')

    # Does the patient eventually have sepsis?
    if np.any(labels):
        is_septic = True
        t_sepsis = np.argmax(labels) - dt_optimal
    else:
        is_septic = False
        t_sepsis = float('inf')

    n = len(labels)

    # Define slopes and intercept points for utility functions of the form
    # u = m * t + b.
    m_1 = float(max_u_tp) / float(dt_optimal - dt_early)
    b_1 = -m_1 * dt_early
    m_2 = float(-max_u_tp) / float(dt_late - dt_optimal)
    b_2 = -m_2 * dt_late
    m_3 = float(min_u_fn) / float(dt_late - dt_optimal)
    b_3 = -m_3 * dt_optimal

    # Compare predicted and true conditions.
    u = np.zeros(n)
    for t in range(n):
        if t <= t_sepsis + dt_late:
            # TP
            if is_septic and predictions[t]:
                if t <= t_sepsis + dt_optimal:
                    u[t] = max(m_1 * (t - t_sepsis) + b_1, u_fp)
                elif t <= t_sepsis + dt_late:
                    u[t] = m_2 * (t - t_sepsis) + b_2
            # FP
            elif not is_septic and predictions[t]:
                u[t] = u_fp
            # FN
            elif is_septic and not predictions[t]:
                if t <= t_sepsis + dt_optimal:
                    u[t] = 0
                elif t <= t_sepsis + dt_late:
                    u[t] = m_3 * (t - t_sepsis) + b_3
            # TN
            elif not is_septic and not predictions[t]:
                u[t] = u_tn

    # Find total utility for patient.
    return np.sum(u)

In [7]:
compute_prediction_utility([1],[1])

1.0

In [ ]:
i = Input(batch_shape=(1, None, 35))

o = TCN(nb_filters=30, kernel_size=3, nb_stacks=2, dilations=[1, 2, 4, 8, 16, 32], padding='causal', use_skip_connections=True, dropout_rate=0.0, return_sequences=True, name='tcn')(i)
o = Dense(1,activation='sigmoid')(o)

model = Model(inputs=[i], outputs=[o])
model.compile(optimizer='adam',loss='mean_squared_error', metrics=['binary_accuracy'],sample_weight_mode='temporal')

model.fit_generator(gen, epochs=5, steps_per_epoch=len(x_train), max_queue_size=100, verbose=1)

len(y_test)

In [10]:
seps_test_cases=list()
for i in range(len(y_test)):
    if y_test[i][-1,-1]==1:
        seps_test_cases.append(i)
    

In [ ]:
len(seps_test_cases)

In [ ]:
compute_prediction_utility([1],[1])

In [19]:
keras.backend.clear_session()

In [11]:
l=seps_test_cases[1]
pred=((model.predict(np.expand_dims(x_test[l]/(x_test[l].max(axis=0)+0.00000001),axis=0))[0])>.5)*1
actual=y_test[l]
np.hstack([pred,actual])

InternalError: Blas SGEMM launch failed : m=84, n=20, k=35
	 [[Node: conv1d_266/convolution/Conv2D = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](conv1d_266/convolution/ExpandDims, conv1d_266/convolution/ExpandDims_1)]]
	 [[Node: dense_11/Sigmoid/_1155 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_840_dense_11/Sigmoid", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'conv1d_266/convolution/Conv2D', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launche

In [ ]:
x_test[131].shape

In [8]:
from keras.models import load_model

model.save('TCN_00.h5')  # creates a HDF5 file 'my_model.h5'

NameError: name 'model' is not defined

In [20]:
from keras.models import load_model
model = load_model('TCN_00.h5')

InternalError: GPU sync failed

In [ ]:
model.predict(y_)

In [ ]:
seps_test[0:20]

In [ ]:
l=seps_test[13]
shift=8
pred=((model.predict(np.expand_dims(x_test[l][:-shift]/(x_test[l][:-shift].max(axis=0)+0.00000001),axis=0))[0])>.5)*1
actual=y_test[l][shift:]
np.hstack([pred,actual])

In [ ]:
del final_test

In [ ]:
preds_test=list()
final_test=list()
for i in seps_test:
    l=i
    shift=8
    preds_test.append((((model.predict(np.expand_dims(x_test[l][:-shift]/(x_test[l][:-shift].max(axis=0)+0.00000001),axis=0))[0])>.5)*1)[-1])
    final_test.append(y_test[l][shift:][-1])


In [ ]:
sump=0
for i in final_test:
    sump+=i
print(sump)

In [ ]:
sumt=0
for i in preds_test:
    sumt+=i
print(sumt)
    

In [ ]:
(np.asarray(final_test)).shape

In [ ]:
from sklearn.metrics import average_precision_score

In [ ]:
seps_test=list()
for i in range(len(y_test)):
    if y_test[i][-1,-1]==1:
        seps_test.append(i)